## Implementing TimeSeriesRandomClassifier
Open sklearn `RandomForestClassifier` to accept pipelines as `base_estimator`, this required to 
* create `RandomIntervalFeatureExtractor` transformer
* change how parameters are set in construction of individual member of ensemble 
* pass on random state from top-level to all components of pipeline with randomness, including modifying `Pipeline` to have random_state attribute
* adapt sklearn helper functions for parallel building and accumulating predictions of trees
* remove sklearn input checks for tabular data in parent classes

Still to do:
* write unit tests
* ideally replace tabular-data input checks with time-series/panel checks, possibly implemented as part of the data-container, also may be worthwile including keyword argument in pipeline to sidestep input checks in members of ensembles similar to check_input in sklearn `DecisionTreeClassifier`
* `feature_importances_` has to be removed, or adapted to a temporal importance curve (as in the paper)
* `decision_path()` has to be adapted
* `oob_score` methods have to be removed/adapted
* `apply` method has to be removed/adapted
* speed up transfoms, still very slow
* implement "entrance" criterion (as in paper)

In [1]:
from sktime.tsforest import TimeSeriesForestClassifier
from sktime.load_data import load_from_web_to_xdataframe
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
cache_path = 'data/'
dataset_name = 'GunPoint'
X_train, y_train = load_from_web_to_xdataframe(dataset_name, is_train_file=True,
                                               cache_path=cache_path) 
X_test, y_test = load_from_web_to_xdataframe(dataset_name, is_test_file=True,
                                             cache_path=cache_path)
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)
X_test = pd.DataFrame(X_test)
y_test = pd.Series(y_test)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(50, 1) (50,) (150, 1) (150,)


In [7]:
from sktime.tsforest import TimeSeriesForestClassifier
clf = TimeSeriesForestClassifier(n_estimators=100, criterion='entropy')
%time clf.fit(X_train, y_train)
%time clf.score(X_test, y_test)

CPU times: user 15 s, sys: 16.4 ms, total: 15 s
Wall time: 15 s
CPU times: user 42.5 s, sys: 67.9 ms, total: 42.6 s
Wall time: 42.6 s


0.8733333333333333

In [5]:
clf = TimeSeriesForestClassifier(n_estimators=100, criterion='entropy', n_jobs=-1)
%time clf.fit(X_train, y_train)
%time clf.score(X_test, y_test)

CPU times: user 15.6 s, sys: 384 ms, total: 16 s
Wall time: 15.7 s
CPU times: user 43.9 s, sys: 824 ms, total: 44.7 s
Wall time: 43.9 s


0.84